In [11]:
import requests

# Define the API URL
url = "https://ressources.data.sncf.com/api/records/1.0/search/"

# Define the parameters
params = {
    "dataset": "emission-co2-perimetre-complet",  # Ensure dataset is specified
    "q": "",  # Empty query to fetch all records
    "rows": 100,  # Adjust the number of records as needed
    "select": "origine, destination, transporteur, distance_entre_les_gares, "
              "train_empreinte_carbone_kgco2e, autocar_longue_distance_empreinte_carbone_kgco2e, avion_empreinte_carbone_kgco2e, "
              "voiture_electrique_2_2_pers_empreinte_carbone_kgco2e, voiture_thermique_2_2_pers_empreinte_carbone_kgco2e",  # Select relevant fields
}

# Make the API request
response = requests.get(url, params=params)

# Check the response status
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    # Print or process the records
    if "records" in data and data["records"]:
        for record in data["records"]:
            print(record.get("fields", {}))
    else:
        print("No records found.")
else:
    print(f"Error: {response.status_code} - {response.text}")


{'destination_uic': '87746008', 'distance_entre_les_gares': 545.0, 'transporteur': 'TGV', 'origine': 'Paris Gare de Lyon', 'train_empreinte_carbone_kgco2e': 1.5805, 'origine_uic': '87686006', 'destination': 'Annecy', 'autocar_longue_distance_empreinte_carbone_kgco2e': 19.32, 'voiture_electrique_2_2_pers_empreinte_carbone_kgco2e': 26.32, 'voiture_thermique_2_2_pers_empreinte_carbone_kgco2e': 60.368}
{'destination_uic': '87474007', 'distance_entre_les_gares': 249.0, 'transporteur': 'TER', 'origine': 'Rennes', 'train_empreinte_carbone_kgco2e': 6.0756, 'origine_uic': '87471003', 'destination': 'Brest', 'autocar_longue_distance_empreinte_carbone_kgco2e': 8.418, 'voiture_electrique_2_2_pers_empreinte_carbone_kgco2e': 11.468, 'voiture_thermique_2_2_pers_empreinte_carbone_kgco2e': 26.3032}
{'destination_uic': '85030007', 'distance_entre_les_gares': 614.0, 'avion_empreinte_carbone_kgco2e': 70.36030000000001, 'transporteur': 'International', 'origine': 'Paris Gare de Lyon', 'train_empreinte_carb